In [1]:
### import libaries
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

## save to file
with open('hamlet.txt', 'w') as f:
    f.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## load the dataset
with open('hamlet.txt', 'r') as file:
  text = file.read().lower()

## tokenize the text - create index for word
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
print(total_words)

4818


In [3]:
## create input sequences
inputsequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_seq = token_list[:i+1]
    inputsequences.append(n_gram_seq)

## padding sequences
max_seq_len = max([len(x) for x in inputsequences])
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_seq_len,padding='pre'))
inputsequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [4]:
### create predictors and label
import tensorflow as tf
x,y = inputsequences[:,:-1],inputsequences[:,-1]

In [5]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

In [8]:
## train lstm model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Embedding, LSTM, Dense, Dropout

## define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_seq_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

##compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
## define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [12]:
history =model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test),verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 47s 73ms/step - accuracy: 0.0549 - loss: 6.1351 - val_accuracy: 0.0474 - val_loss: 6.9204
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 49s 76ms/step - accuracy: 0.0551 - loss: 5.9292 - val_accuracy: 0.0497 - val_loss: 7.0064
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 47s 72ms/step - accuracy: 0.0592 - loss: 5.8207 - val_accuracy: 0.0527 - val_loss: 7.0751
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 46s 72ms/step - accuracy: 0.0694 - loss: 5.6769 - val_accuracy: 0.0587 - val_loss: 7.1400
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 72ms/step - accuracy: 0.0750 - loss: 5.5086 - val_accuracy: 0.0639 - val_loss: 7.1862
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 48s 75ms/step - accuracy: 0.0809 - loss: 5.3826 - val_accuracy: 0.0670 - val_loss: 7.2655
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 46s 71ms/step - accuracy: 0.0896 - loss: 5.2160 - val_accuracy: 0.0676 - val_loss: 7.3577
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 46s 71ms/step - accuracy: 0.0965 - loss: 5.0961 - 

In [13]:
## function to predict the next word

def predict_next_word(model, tokenizer, text, max_sequence_len):
  token_list = tokenizer.texts_to_sequences([text])[0]
  if len(token_list) >= max_seq_len:
    token_list = token_list[-(max_seq_len-1):]
  token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
  predicted = model.predict(token_list, verbose=0)
  predicted_word_index = np.argmax(predicted, axis=1)
  for word, index in tokenizer.word_index.items():
    if index == predicted_word_index:
      return word
  return None

In [14]:
input_text = "To be or not to be"
print(f"input text: {input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Next word prediction: {next_word}")

input text: To be or not to be
Next word prediction: in't


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Define a permanent path
permanent_path = '/content/drive/MyDrive/Colab Notebooks/lstm.keras'

# Save the model to the permanent location
model.save(permanent_path)

import pickle
# Ensure the folder 'Colab_Notebooks' exists in your Google Drive
save_path = '/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle'

with open(save_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"File saved successfully to: {save_path}")

File saved successfully to: /content/drive/MyDrive/Colab Notebooks/tokenizer.pickle
